In [1]:
from keras.models import Sequential
from keras.layers import Dense, Dropout
import numpy as np
import sys
import import_ipynb
import quickplay
import traceback
#traceback.print_stack()


Using TensorFlow backend.


importing Jupyter notebook from quickplay.ipynb


In [6]:
def log_traceback(ex, sim):
    tb_lines = traceback.format_exception(ex.__class__, ex, ex.__traceback__)
    tb_text = ''.join(tb_lines)
    # I'll let you implement the ExceptionLogger class,
    # and the timestamping.
    print(tb_text)
    print("info: ")
    print("\t",sim.p1.turn)
    print("\t",sim.shop.cards)
    print("\t",sim.shop.checkEnd())
    print("\tp1")
    print("\t",sim.p1.hand)
    print("\t",sim.p1.deck)
    print("\t",sim.p1.discard)
    print("\tp2")
    print("\t",sim.p2.hand)
    print("\t",sim.p2.deck)
    print("\t",sim.p2.discard)



In [7]:
from datetime import datetime

start = datetime.now()

x=quickplay.Simulation()
wins = [0]*100
for i in range(len(wins)):
    j=0
    while True:
        j+=1
        if x.p1.turn == x.p2.turn:
            player = x.p1
        else: 
            player = x.p2
        try:
            new_s, reward, end = x.makeDecision(player,1)
        except Exception as ex:
            log_traceback(ex, x)
            sys.exit()
        if end:
            break
            
    if x.p1.getVP() > x.p2.getVP():
        wins[i]=1
    else:
        wins[i]=2
    x=quickplay.Simulation()
#print(str(wins))
finish = datetime.now()
print("Time =", str(finish-start))
print(wins.count(1))
print(wins.count(2))




Time = 0:00:00.515257
47
53


In [8]:
model = Sequential()
model.add(Dense(10, activation='sigmoid', input_shape=(18,)))
model.add(Dense(2, activation='linear'))
model.compile(loss='mse', optimizer='adam', metrics=['mae'])

In [9]:
y = 0.95
eps = 0.5
decay_factor = 0.999
game_lengths = []
num_episodes = 200
for i in range(num_episodes):
    sim = quickplay.Simulation()
    eps *= decay_factor
    if i % 9 == 0:
        print("Episode {} of {}".format(i + 1, num_episodes))
    done = False
    turns = 0
    while not done:
        if x.p1.turn == x.p2.turn:
            player = x.p1
        else: 
            player = x.p2
        current_state = np.array(sim.getBuyState(player))
        if np.random.random() < eps:
            action = np.random.randint(0, 2)
        else:
            action = np.argmax(model.predict(current_state))
        try:
            new_state, reward, done = sim.makeDecision(player, action)
            new_state = np.array(new_state)
        except Exception as ex:
            log_traceback(ex, sim)
            sys.exit()
        target = reward + y * np.max(model.predict(new_state))
        target_vec = model.predict(current_state)[0]
        target_vec[action] = target
        model.fit(current_state, target_vec.reshape(-1, 2), epochs=1, verbose=0)
    game_lengths.append(sim.p1.turn)

Episode 1 of 50
Episode 6 of 50
Episode 11 of 50
Episode 16 of 50
Episode 21 of 50
Episode 26 of 50
Episode 31 of 50
Episode 36 of 50
Episode 41 of 50
Episode 46 of 50


In [15]:
print(game_lengths)

2

In [13]:
print(np.shape(sim.getBuyState(player)))
print(sim.getBuyState(player))

(1, 18)
[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 7, 0, 0, 3, 0, 0, 1, 3]]


In [11]:
arrTest1 = np.array([[0.1, 0.1, 0.1, 0.1, 0.1, 0.5, 0.1, 0., 0.1, 0.6, 0.1, 0.1, 0., 0., 0., 0.1, 0., 0., 0.1, 0., 0.]])

(1, 21)
2
